In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sysgen import sysgen
from sysid import sysid
from syssim import syssim

In [39]:
# System Data
n = 3
p = 2

# Nominal system:
A_0 = np.array([[0.6, 0.5, 0.4],
                [0, 0.4, 0.3],
                [0, 0, 0.3]])

B_0 = np.array([[1, 0.5],
                [0.5, 1],
                [0.5, 0.5]])

V = np.array([[0, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])  # Modification pattern applied to A_0

U = np.array([[1, 0],
              [0, 0],
              [0, 1]])  # Modification pattern applied to B_0

# Noise levels
sigu = 1
sigw = 1
sigx = 1

# Rollout length
T = 5

# Select FL_solver (Fed_Avg = 0, Fed_Lin = 1)
FL_solver = 0

q = 25  # number of estimations
R = 200  # number of global iterations
s = 1  # fixed system for the error computation

M = [1, 2, 5, 25, 100] 
N = 25  # Fixed number of rollouts
epsilon = 0.01  # Fixed dissimilarity

In [40]:
# Generating the system matrices
# Define `sysgen` to generate similar systems
A, B = sysgen(A_0, B_0, V, U, M, epsilon)

E_avg = np.zeros((len(M), R))

# Numerical results varying the number of clients
Error_matrix = np.zeros((q, R))


In [41]:
true_theta = np.hstack([A_0, B_0])
true_theta

array([[0.6, 0.5, 0.4, 1. , 0.5],
       [0. , 0.4, 0.3, 0.5, 1. ],
       [0. , 0. , 0.3, 0.5, 0.5]])

In [42]:
M=100
R=200

In [38]:

n = A[0].shape[0]
p = B[0].shape[1]
s=0


X = []
Z = []
W = []
for i in range(M):
    X_i, Z_i, W_i, list_x0 = syssim(A, B, T, N, i, sigu, sigw, sigx)
    X.append(X_i)
    Z.append(Z_i)
    W.append(W_i)

# FedSysID

# Initialiser le serveur avec \bar{\Theta}_0 et \alpha
Theta_0 = np.hstack([(1/2) * A[s], (1/2) * B[s]])
alpha = 1e-4  # pas d'apprentissage
K = 10  # nombre d'itérations locales


Theta_s = Theta_0.copy()  # serveur
Theta_c = [None] * M  # clients
print(f"longueur de Theta_c : {len(Theta_c)} \n")
Error = np.zeros(R) # vecteur d'erreur  de dim (R,)

for r in range(R):
       
    print(f"Round {r} \n")

    print(f"valeur de theta_s en début de round : \n{Theta_s} \n")

    # Initialiser chaque client avec \bar{\Theta}_0
    for i in range(M):
        Theta_c[i] = Theta_s.copy()
    
    # Côté client :
    for i in range(M):
        print(f"Coté client, boucle, i vaut : {i} \n")
        if FL_solver == 0:
            # FedAvg
            for k in range(1, K + 1):
                # tmp= Theta_c[i]
                # print(f"dimension de Theta_c[i] vaut : {tmp.shape} \n")
                new_val = Theta_c[i] + (alpha / k) * ((X[i] - Theta_c[i] @ Z[i]) @ Z[i].T)
                # print(f"dimension de new_val : {new_val.shape} \n")
                Theta_c[i] = new_val

    # # Côté serveur :
    Theta_sum = np.zeros((n, n + p))
    for i in range(M):
        Theta_sum = Theta_sum + Theta_c[i]
    Theta_s = (1 / M) * Theta_sum
        

    print(f"valeur de theta_s en fin de round : \n{Theta_s} \n")
    print(f"FIN Round {r} \n ======================== \n")
    Error[r] = np.linalg.norm(Theta_s - true_theta, ord=2) # TODO : askip ici l'erreur c la norme 2 et jsp pk  # DONE : ord=2 impose norme spectale


longueur de Theta_c : 25 

Round 0 

valeur de theta_s en début de round : 
[[0.3        0.25       0.2        0.50292638 0.25      ]
 [0.         0.20173979 0.15       0.25       0.5       ]
 [0.         0.         0.15173979 0.25       0.25292638]] 

Coté client, boucle, i vaut : 0 

Coté client, boucle, i vaut : 1 

Coté client, boucle, i vaut : 2 

Coté client, boucle, i vaut : 3 

Coté client, boucle, i vaut : 4 

Coté client, boucle, i vaut : 5 

Coté client, boucle, i vaut : 6 

Coté client, boucle, i vaut : 7 

Coté client, boucle, i vaut : 8 

Coté client, boucle, i vaut : 9 

Coté client, boucle, i vaut : 10 

Coté client, boucle, i vaut : 11 

Coté client, boucle, i vaut : 12 

Coté client, boucle, i vaut : 13 

Coté client, boucle, i vaut : 14 

Coté client, boucle, i vaut : 15 

Coté client, boucle, i vaut : 16 

Coté client, boucle, i vaut : 17 

Coté client, boucle, i vaut : 18 

Coté client, boucle, i vaut : 19 

Coté client, boucle, i vaut : 20 

Coté client, boucle, i

In [34]:
Error

array([0.88267599, 0.83264077, 0.79066974, 0.75416395, 0.72148757,
       0.69160277, 0.66384321, 0.63777444, 0.61310777, 0.58964676,
       0.56725369, 0.5458286 , 0.52529596, 0.50559626, 0.48668072,
       0.46850785, 0.45104127, 0.4342483 , 0.41809907, 0.40256588,
       0.38762284, 0.37324556, 0.35941098, 0.3460972 , 0.33328342,
       0.32094979, 0.30907743, 0.29764827, 0.28664508, 0.27605141,
       0.26585149, 0.25603028, 0.24657337, 0.237467  , 0.22869797,
       0.22025367, 0.21212203, 0.20429148, 0.19675096, 0.18948988,
       0.18249811, 0.17576596, 0.16928415, 0.16304382, 0.1570365 ,
       0.15125414, 0.14568903, 0.14033386, 0.13518167, 0.13022591,
       0.12546039, 0.1208793 , 0.11647725, 0.11224928, 0.10819091,
       0.10429814, 0.10056756, 0.09699644, 0.09358277, 0.09032546,
       0.08722448, 0.08428102, 0.08149771, 0.07887879, 0.07643007,
       0.07415879, 0.07207283, 0.07017943, 0.06848318, 0.06698386,
       0.0656749 , 0.06454318, 0.06357058, 0.06273644, 0.06201